In [15]:
from PIL import Image, ImageDraw, ImageFont
import os

In [16]:
pip install cairosvg


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [25]:
# Image set from: https://github.com/tobecwb/stable-diffusion-face-dataset/tree/main

import os
from io import BytesIO
from PIL import Image
from cairosvg import svg2png

INPUT_DIR = "ai-validated"
OUTPUT_DIR = "ai_labeled_human"
SVG_LABEL_PATH = "label_human.svg"

LABEL_HEIGHT_RATIO = 0.05  # base ratio of image height
LABEL_SCALE = 1.5          # scale multiplier for label size (1.0 = normal, 2.0 = double)
MARGIN = 1


def convert_svg_to_png(svg_path, target_height):
    with open(svg_path, "rb") as f:
        svg_bytes = f.read()
    png_bytes = svg2png(bytestring=svg_bytes, output_height=target_height)
    return Image.open(BytesIO(png_bytes)).convert("RGBA")


def add_svg_label(image_path, svg_img, output_dir, margin):
    img = Image.open(image_path).convert("RGBA")
    w, h = img.size

    # Scale label relative to image height and scale factor
    target_label_h = int(h * LABEL_HEIGHT_RATIO * LABEL_SCALE)
    aspect = svg_img.width / svg_img.height
    label_resized = svg_img.resize(
        (int(target_label_h * aspect), target_label_h),
        resample=Image.LANCZOS
    )

    lw, lh = label_resized.size

    # Bottom-right corner with margin
    x = w - lw - margin
    y = h - lh - margin

    base = Image.new("RGBA", img.size)
    base = Image.alpha_composite(base, img)
    base.alpha_composite(label_resized, dest=(x, y))

    os.makedirs(output_dir, exist_ok=True)
    name, _ = os.path.splitext(os.path.basename(image_path))
    output_path = os.path.join(output_dir, f"{name}_labeled.png")
    base.convert("RGB").save(output_path)
    print(f"Labeled: {output_path}")


def process_folder():
    svg_img = convert_svg_to_png(SVG_LABEL_PATH, target_height=200)

    supported_exts = {".jpg", ".jpeg", ".png"}
    files = [f for f in os.listdir(INPUT_DIR)
             if os.path.splitext(f)[1].lower() in supported_exts]

    for file in files:
        img_path = os.path.join(INPUT_DIR, file)
        try:
            add_svg_label(img_path, svg_img, OUTPUT_DIR, margin=MARGIN)
        except Exception as e:
            print(f"Failed to process {file}: {e}")


if __name__ == "__main__":
    process_folder()


Labeled: ai_labeled_human/man_0713_labeled.png
Labeled: ai_labeled_human/man_1140_labeled.png
Labeled: ai_labeled_human/man_0516_labeled.png
Labeled: ai_labeled_human/man_0356_labeled.png
Labeled: ai_labeled_human/man_0886_labeled.png
Labeled: ai_labeled_human/woman_0249_labeled.png
Labeled: ai_labeled_human/men_1470_labeled.png
Labeled: ai_labeled_human/man_0633_labeled.png
Labeled: ai_labeled_human/man_0899_labeled.png
Labeled: ai_labeled_human/man_0887_labeled.png
Labeled: ai_labeled_human/woman_0276_labeled.png
Labeled: ai_labeled_human/woman_0460_labeled.png
Labeled: ai_labeled_human/men_0064_labeled.png
Labeled: ai_labeled_human/man_0608_labeled.png
Labeled: ai_labeled_human/man_0963_labeled.png
Labeled: ai_labeled_human/woman_0446_labeled.png
Labeled: ai_labeled_human/woman_0502_labeled.png
Labeled: ai_labeled_human/man_0993_labeled.png
Labeled: ai_labeled_human/man_0164_labeled.png
Labeled: ai_labeled_human/man_0597_labeled.png
Labeled: ai_labeled_human/man_0592_labeled.png
Lab